# Init

In [290]:
#import
import numpy as np 

# Variable
eta = 0.9;

xlist = [];
dList = [];

eList = []
biasList = [];
wMatrix = [];
vMatrix = []
yMatrix = [];
sigmaMatrix = [];
nodeIndexMatrix = [];

## Functions

### Auxilary Function

In [291]:
def getNodeLayer(nodeNum:int):
    result = 0;
    for layer in nodeIndexMatrix:
        if nodeNum in layer: return result
        result += 1
    raise RuntimeError("no node " + nodeNum + "existed in nodeIdexMatrix");

def getNodeLayerNIndex(nodeNum:int):
    nodeLayer = getNodeLayer(nodeNum)
    nodeIndex = nodeIndexMatrix[nodeLayer].index(nodeNum)
    return nodeLayer, nodeIndex

### Calculation Function

In [292]:
def vFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0:
        return 0;
    else:
        pastIndexes = nodeIndexMatrix[nodeLayer-1];
        pastWs = list(map(lambda x: wMatrix[x][nodeIndex], pastIndexes))
        pastYs = yMatrix[nodeLayer-1]
        vlist = list(map(lambda x,y: x*y, pastYs,pastWs))
        bias = biasList[nodeNum];
        v = round(sum(vlist)+bias,3);
        return v;


def simgFunc(input:int):
    return 1/(1 + np.exp(-input));

def simgPrimeFunc(input:int):
    return simgFunc(input)*(1-simgFunc(input));

def yFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0:
        return xlist[nodeNum];
    else:
        v = vMatrix[nodeLayer][nodeIndex];
        return round(simgFunc(v),3);

def eFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1:
        y = yMatrix[nodeLayer][nodeIndex]
        d = dList[nodeIndex]
        return d - y;
    else:
        raise RuntimeError("Can't calculate e("+str(nodeNum)+")")

def sigmaFunc(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    phiPrime = simgPrimeFunc(vFunc(nodeNum))
    if nodeLayer == len(nodeIndexMatrix)-1:
        e = eFunc(nodeNum);
        return round(e*phiPrime,4);
    else:
        forWs = wMatrix[nodeNum];
        forSigmas = sigmaMatrix[nodeLayer+1]
        sumSigW = sum(map(lambda x,y: x*y,forWs, forSigmas));
        return round(phiPrime*sumSigW,4);

def deltaWFunc(nodeI:int, nodeJ:int):
    nodeILayer, nodeIIndex = getNodeLayerNIndex(nodeI);
    nodeJLayer, nodeJIndex = getNodeLayerNIndex(nodeJ);
    yI = yMatrix[nodeILayer][nodeIIndex];
    if nodeJLayer == len(nodeIndexMatrix)-1:
        e = eFunc(nodeJ);
        phiPrime = simgPrimeFunc(vFunc(nodeJ))
        return round(eta*e*phiPrime*yI,3);
    else:
        sigmaJ = sigmaMatrix[nodeJLayer][nodeJIndex];
        return round(eta*sigmaJ*yI,3);

def getNewWs(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == len(nodeIndexMatrix)-1:
        raise RuntimeError("Cann't calculate delta-Ws of output node:" + str(nodeNum))
    forNodeIndexes = nodeIndexMatrix[nodeLayer+1];
    oldWs = wMatrix[nodeNum]
    deltaWs = list(map(lambda x: deltaWFunc(nodeNum, x), forNodeIndexes))
    newWs = list(map(lambda x,y: x+y, oldWs,deltaWs))
    return newWs;

def getNewBias(nodeNum:int):
    nodeLayer, nodeIndex = getNodeLayerNIndex(nodeNum);
    if nodeLayer == 0: raise RuntimeError("No bias for node:"+str(nodeNum))
    sigma = sigmaMatrix[nodeLayer][nodeIndex];
    oldBias = biasList[nodeNum];
    newBias = oldBias+(eta*sigma);
    return round(newBias,3);



# Training Ground

## Training Function

In [293]:
def initAllContainer(layerNodeCount:list):
    #Clear
    xlist.clear();
    dList.clear();
    eList.clear();
    biasList.clear();
    wMatrix.clear();
    vMatrix.clear()
    yMatrix.clear();
    sigmaMatrix.clear();
    nodeIndexMatrix.clear();
    #Create 
    currIndex = 0;
    for l in layerNodeCount:
        nodeIndexMatrix.append(list(range(currIndex,currIndex+l)))
        yMatrix.append([0]*l)
        vMatrix.append([0]*l)
        sigmaMatrix.append([0]*l)
        currIndex += l
    for layer in range(len(nodeIndexMatrix)):
        for node in nodeIndexMatrix[layer]:
            if not node in nodeIndexMatrix[len(nodeIndexMatrix)-1]:
                wMatrix.append([0]*len(nodeIndexMatrix[layer+1]))
    for _ in range(max(max(nodeIndexMatrix))+1):
        eList.append(0);
        biasList.append(0);

def getRandomWList():
    pass;

def getXList(dataIndex:int):
    pass;

def getDList(dataIndex:int):
    pass;

def updateAllY():
    for layNum in range(len(nodeIndexMatrix)):
        nodelist = nodeIndexMatrix[layNum]
        vList = list(map(vFunc, nodelist));
        vMatrix[layNum] = vList
        yList = list(map(yFunc, nodelist));
        yMatrix[layNum] = yList

def backPropergate():
    reverseLayNum = list(range(len(nodeIndexMatrix))).copy()
    reverseLayNum.pop(0)
    reverseLayNum.reverse()
    for layNum in reverseLayNum:
        nodelist = nodeIndexMatrix[layNum]
        sigmaList = list(map(sigmaFunc, nodelist));
        sigmaMatrix[layNum] = sigmaList
    for node in range(max(max(nodeIndexMatrix))):
        newWs = getNewWs(node);
        wMatrix[node] = newWs;
    startBiasIndex = max(nodeIndexMatrix[0])
    for b in range(startBiasIndex+1,max(max(nodeIndexMatrix))+1):
        newBias = getNewBias(b);
        biasList[b] = newBias;


## Execution

In [294]:
# input attribute value

# calculate hidden layer one

# calculate final layer

# keep set of results 

# Evaluation

In [295]:
initAllContainer([3,2,1])
wMatrix = [[0.2,-0.3],[0.4,0.1],[-0.5,0.2],[-0.3],[-0.2]]
xlist = [1,0,1]
dList = [1]
biasList = [0,0,0,-0.4,0.2,0.1]
# # print(vMatrix);
# # print(yMatrix);
updateAllY();
# print(wMatrix)
# print(biasList)

backPropergate();
print(sigmaMatrix)
print(wMatrix)
print(biasList)

[[0, 0, 0], [-0.0087, -0.0065], [0.1311]]
[[0.192, -0.306], [0.4, 0.1], [-0.508, 0.194], [-0.261], [-0.138]]
[0, 0, 0, -0.408, 0.194, 0.218]
